<a href="https://colab.research.google.com/github/z-arabi/pytorchTutorial/blob/master/08_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

Linear regression > for predicting one value  
Logistic regression > for classification  
classification > separable or not separable  
linear separable OR non-linear seperable  
Log reg > fits best for linear separable data  
predict a line which separates the classes  
Log reg = lin reg + function

In [5]:
# 0) Prepare data
bc = datasets.load_breast_cancer()

print(bc.keys())
print(bc.data[0],bc.target[0])
print(bc.data[0].shape)

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])
[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01] 0
(30,)


In [7]:
X, y = bc.data, bc.target
print(X.shape,y.shape)

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

(569, 30) (569,)


In [8]:
# scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
# always converts to float32 first > then tensor
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [10]:
# 1) Model
# in reg always we have one output > the output dim is 1
# Linear model f = wx + b , sigmoid at the end
class Model(nn.Module):
    def __init__(self, n_input_features):
        super(Model, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = Model(n_features)

In [14]:
# 2) Loss and optimizer
num_epochs = 1000
learning_rate = 0.01

# binary cross entrophy
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
# 3) Training loop
for epoch in range(num_epochs):
    # Forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)

    # Backward pass and update
    loss.backward()
    optimizer.step()

    # zero grad before new step
    optimizer.zero_grad()

    if (epoch+1) % 100 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')

epoch: 100, loss = 0.0828
epoch: 200, loss = 0.0803
epoch: 300, loss = 0.0782
epoch: 400, loss = 0.0763
epoch: 500, loss = 0.0745
epoch: 600, loss = 0.0729
epoch: 700, loss = 0.0715
epoch: 800, loss = 0.0701
epoch: 900, loss = 0.0689
epoch: 1000, loss = 0.0677


In [19]:
# find the accuracy of the model mabually
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    print(y_predicted_cls.eq(y_test)[:5])
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

tensor([[True],
        [True],
        [True],
        [True],
        [True]])
accuracy: 0.9386
